In [ ]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="1"
import gc

import numba
import numpy as np
import numpy.ma as ma
import pandas as pd

import matplotlib.pyplot as plt

from sklearn import mixture
from scipy import linalg as LA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from multiprocessing import Pool
import time

import glob
import h5py
from scipy import sparse
from scipy import ndimage

from MulticoreTSNE import MulticoreTSNE as TSNE

import scipy.cluster as cl
import scipy.spatial.distance as ssd
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram
import tensorflow as tf

In [ ]:
from utils import compute_wavelet

In [ ]:
file_name = '<>'
DATA_DIR = '<>/'
OUTPUT_DIR = '<>/'
data_file = os.path.join(DATA_DIR, file_name)

In [ ]:
df = pd.DataFrame()

features = compute_wavelet(df)